# Classifier
This is more of a first configuration just to try out and make sure it is up and running.
1. Convolutional layer (unit strides) + ReLu
2. Pooling layer
- Convolutional layer (unit strides) + ReLu
- Pooling layer
- Fully connected layer with dropout
- Fully connected layer
- Softmax activation

My knowledge on the best practices of neural networks is not extensive and I'd appreciate some feedback. I referred to [this blog post](https://dev.to/kasperfred/putting-it-all-together---introduction-to-tensorflow-part-5-ein) and this [official page](https://www.tensorflow.org/tutorials/layers).

In [1]:
import tensorflow as tf
import pickle
import numpy as np

/home/bgalvao/miniconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('dataset.npys', 'rb') as handle:
    train, train_labels, test, test_labels = pickle.load(handle)
print(train.shape)

(308, 50, 50, 3)


A side note, to be able to tell what size is produced out of a convolutional layer, with zero-padding on the edges, non-unit stride, and following this [awesome guide](https://arxiv.org/abs/1603.07285v1):
$$o_c = \frac{i + 2p -k}{s}+1$$
where $i$ is the size of the (square) input, $p$ is the amount of zero-padding, $k$ is the size of the (square) kernel and $s$ is the stride. For the pooling layer, the resulting size is described by:
$$o_p = \frac{i-k}{s}+1$$

In [10]:
tf.reset_default_graph()
with tf.name_scope('input') as scope:
    x = tf.placeholder(tf.float32, [None, 50, 50, 3], name='input')
    labels = tf.placeholder(tf.float32, [None, 2], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

with tf.name_scope('model') as scope:

    with tf.name_scope('convpool1') as scope:
        conv1 = tf.layers.conv2d(
            inputs = x,
            filters = 32,
            kernel_size = [6, 6],
            padding = 'valid',
            activation = tf.nn.relu,
        )
        print('conv1 ::', conv1.shape)
        
        pool1 = tf.layers.max_pooling2d(
            inputs=conv1,
            pool_size=[2,2],
            strides=2
        )
        print('pool1 ::', pool1.shape)
        
    with tf.name_scope('convpool2') as scope:
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[3,3],
            padding='valid',
            activation=tf.nn.relu
        )
        print('conv2 ::', conv2.shape)
        
        pool2 = tf.layers.max_pooling2d(
            inputs=conv2,
            pool_size=[3,3],
            strides=1
        )
        print('pool2 ::', pool2.shape)
        
    with tf.name_scope('fc1') as scope:
        pool2_flat = tf.reshape(pool2, [-1, 18 * 18 * 64])
        print(pool2_flat.shape)
        dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
        dropout = tf.nn.dropout(dense, keep_prob)
        print('fc1   ::', dropout.shape)
        
    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(inputs=dropout, units=2)
        print('logits ::', logits.shape)
        
with tf.name_scope('train') as scope:
    with tf.name_scope('loss') as scope:
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    training_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.name_scope('evaluation') as scope:
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# create a summarizer that summarizes loss and accuracy
tf.summary.scalar("Accuracy", accuracy)
tf.summary.scalar("Loss", tf.reduce_mean(cross_entropy))
summary_op = tf.summary.merge_all()

# create saver object
saver = tf.train.Saver()

conv1 :: (?, 45, 45, 32)
pool1 :: (?, 22, 22, 32)
conv2 :: (?, 20, 20, 64)
pool2 :: (?, 18, 18, 64)
(?, 20736)
fc1   :: (?, 512)
logits :: (?, 2)


In [15]:
with tf.Session() as sess:
    
    # initialize variables
    tf.global_variables_initializer().run()
    
    # initialize summarizer filewriter
    fw = tf.summary.FileWriter("./log_tensorflow", sess.graph)

    # train the network
    for step in range(200):

        # this should be a batch instead
        sess.run(training_step, feed_dict={
                x: train,
                labels: train_labels,
                keep_prob:1.0
            })
        if step%2 == 0:
            acc = sess.run(accuracy, feed_dict={
                x: train,
                labels: train_labels,
                keep_prob:1.0
            })
            
            acct = sess.run(accuracy, feed_dict={
                x: test,
                labels: test_labels,
                keep_prob:1.0
            })
            print("@step:", step, "train_accuracy:", acc, 'test_accuracy:', acct)

            summary = sess.run(summary_op, feed_dict={
                x: test,
                labels: test_labels,
                keep_prob: 1.0
            })

            fw.add_summary(summary, step)

    # save trained model
    saver.save(sess, "./my.model")

@step: 0 train_accuracy: 0.5 test_accuracy: 0.5
@step: 2 train_accuracy: 0.5 test_accuracy: 0.5
@step: 4 train_accuracy: 0.5 test_accuracy: 0.5
@step: 6 train_accuracy: 0.5 test_accuracy: 0.5
@step: 8 train_accuracy: 0.5 test_accuracy: 0.5
@step: 10 train_accuracy: 0.6136364 test_accuracy: 0.57831323
@step: 12 train_accuracy: 0.7532467 test_accuracy: 0.73493975
@step: 14 train_accuracy: 0.8149351 test_accuracy: 0.8072289
@step: 16 train_accuracy: 0.8214286 test_accuracy: 0.78313255
@step: 18 train_accuracy: 0.8084416 test_accuracy: 0.7590361
@step: 20 train_accuracy: 0.8279221 test_accuracy: 0.8313253
@step: 22 train_accuracy: 0.8084416 test_accuracy: 0.7710843
@step: 24 train_accuracy: 0.85064936 test_accuracy: 0.8313253
@step: 26 train_accuracy: 0.8474026 test_accuracy: 0.8192771
@step: 28 train_accuracy: 0.8474026 test_accuracy: 0.8072289
@step: 30 train_accuracy: 0.8603896 test_accuracy: 0.82530123
@step: 32 train_accuracy: 0.8668831 test_accuracy: 0.813253
@step: 34 train_accuracy

Results to be improved!